<a href="https://colab.research.google.com/github/vincent0965/Colab_note/blob/master/news_kind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
glob.glob("drive/My Drive/*")

In [8]:
import zipfile
path = "drive/My Drive/chinese_news_test.zip"
f = zipfile.ZipFile(path)
f.extractall()
path = "drive/My Drive/chinese_news_trans.zip"
f = zipfile.ZipFile(path)
f.extractall()

In [7]:
import os
import glob
import pandas as pd

def read_data(base):
    contents, targets = [], []
    basedn = os.path.join(base, "*")
    for dn in glob.glob(basedn):
        upper = os.path.join(dn, "*.TXT")
        lower = os.path.join(dn, "*.txt")
        for fn in glob.glob(upper) + glob.glob(lower):
            with open(fn, "r", encoding="utf-8") as f:
                contents.append(f.read())
            target = os.path.split(dn)[-1]
            targets.append(target)
    df = pd.DataFrame({
        "content":contents,
        "target":targets
    })
    return df
train_df = read_data("chinese_news_trans")
test_df = read_data("chinese_news_test")

In [9]:
train_df

,content,target
0,《瞭望》週刊文章：一個全球性的環境問題——垃圾\n新華社北京５月１２日...,環境
1,英國確定溫室效應政策\n新華社倫敦５月２５日電為了對付由於溫室效應引起...,環境
2,【 日 期 】19960610\n【 版 號 】10\n【 標 題 】海南人的藍色樂章...,環境
3,經國務院批准我國長江中上游防護林體系建設工程全\n面展開\n (...,環境
4,【 日 期 】19960105\n【 版 號 】7\n【 標 題 】科學家選出２０種最...,環境
...,...,...
2632,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
2633,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
2634,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
2635,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟


In [10]:
test_df

,content,target
0,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
1,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
2,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
3,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
4,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
...,...,...
96,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
97,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
98,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
99,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟


In [11]:
# Series.replace({"李白":0})
# unique/value_counts
cat = train_df["target"].unique()
trans = {w:i for i, w in enumerate(cat)}
trans_r = {i:w for i, w in enumerate(cat)}
# 維度轉換
y_train = train_df["target"].replace(trans)
y_test = test_df["target"].replace(trans)
y_test

0      0
1      0
2      0
3      0
4      0
      ..
96     9
97     9
98     9
99     9
100    9
Name: target, Length: 101, dtype: int64

In [14]:
import jieba
from urllib.request import urlretrieve
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.txt.big")
jieba.set_dictionary("dict.txt.big")
# apply(函式名字)
def newscut(p):
    return " ".join(jieba.cut(p))
x_train = train_df["content"].apply(newscut)
x_test = test_df["content"].apply(newscut)
x_test

Building prefix dict from /content/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.566 seconds.
Prefix dict has been built successfully.


0      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
1      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
2      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
3      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
4      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
                             ...                        
96     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
97     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
98     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
99     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
100    日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
Name: content, Length: 101, dtype: object

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [16]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.1)
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [17]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

1.0

In [19]:
p = input("輸入新聞:")
x_predict = vec.transform([newscut(p)])
proba = clf.predict_proba(x_predict)[0]
for w, p in zip(cat, proba):
    print(w, ":", round(p, 4))

輸入新聞:美國國務卿龐培歐訪問日本，他接受日媒專訪時透露，有意打造一個「對抗中國共產黨的安全保障網」。被問及中共武力犯台，美方是否會協防台灣？龐培歐表示，美國將採取一切行動，緩解區域的緊張情勢，「坐視中國的行為並非正確答案，不論是對台灣或日本，美國一直是安全保障上的夥伴」。  打造「對抗中共的安全保障網」 龐培歐接受日本經濟新聞專訪時明確表達，有意打造一個「對抗中國共產黨的安全保障網」，希望將美、日、印、澳四國合作制度化後，擴大到周邊其他國家，建立印度太平洋地區的多邊安保框架，強化具有共同價值觀和戰略的國家間的凝聚力，劍指中國。  美國與台灣 有相同的價值觀 台美關係上，他提到，美國與台灣建立長期關係，有著相同的價值觀，但令人遺憾的是，中國共產黨拒絕這樣做，也未讓香港維持五十年高度自治。  中國暴力手段 不是大國作為 被問到若中國單方面攻擊台灣，美國是否準備捍衛台灣？龐培歐表示，美方將採取一切措施緩解區域的緊張局勢。這是川普政府在全球的任務；美方不是要挑起衝突，而是在尋求和平，該感到羞恥的是中國共產黨；除了與越南關係緊張，迫使日方不得不在釣魚台空域頻繁緊急升空因應，還有喜馬拉雅（中印邊境）發生的事情，中國用暴力手段，不是大國應有的所為。  美中對立 是國際社會的問題 龐培歐在接受ＮＨＫ專訪時，對於美中因台、港加深對立，他認為這不是美國對上中國的問題，而是自由與專制政治要選擇哪邊的問題，讓使用軍事或威迫力量欺壓弱者的國家來支配世界好嗎？他強調，美中對立並非雙邊問題，而是國際社會的問題。  龐培歐譴責中國在南海及東海誇耀軍事力量，持續採取威迫性行動。他說，這是緊急課題，全球長期以來被置於中國威脅下，現在是必須認真因應這個問題的時候。
環境 : 0.0
體育 : 0.0
交通 : 0.0
醫藥 : 0.0
軍事 : 0.0
藝術 : 0.0
計算機 : 0.0
教育 : 0.0
政治 : 1.0
經濟 : 0.0
